<a href="https://colab.research.google.com/github/bhavana-kedari/Stop-Sign-detection_OpenCV/blob/master/PCAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
! pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2sodil93
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2sodil93
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=af47975edc34e8a8e0613f2b2b7755a4424bd2c7558a6cf6e6149784bc77fdd3
  Stored in directory: /tmp/pip-ephem-wheel-cache-htu6tayo/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%cd /content

/content


In [6]:
!rm -r PCAP-AIML-2023

rm: cannot remove 'PCAP-AIML-2023': No such file or directory


In [7]:
!git clone https://github.com/PRAKASH-KALINGRAO-AITHAL/PCAP-AIML-2023.git

Cloning into 'PCAP-AIML-2023'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 118 (delta 49), reused 66 (delta 1), pack-reused 0
Receiving objects: 100% (118/118), 571.94 KiB | 5.25 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [8]:
%cd PCAP-AIML-2023


/content/PCAP-AIML-2023


In [9]:
!ls

Lena.jpeg  README.md  rgbToGray  rgbToGray.sln	stb_image.h  stb_image_resize.h  stb_image_write.h


In [10]:
!/content/src/pcap.out

/bin/bash: line 1: /content/src/pcap.out: No such file or directory


In [11]:
!apt-get --purge remove cuda
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get -y install cuda

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'cuda' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
dpkg: dependency problems prevent removal of cuda-cccl-11-8:
 cuda-minimal-build-11-8 depends on cuda-cccl-11-8 (>= 11.8.89).
 cuda-libraries-dev-11-8 depends on cuda-cccl-11-8 (>= 11.8.89).
 cuda-cudart-dev-11-8 depends on cuda-cccl-11-8.

dpkg: error processing package cuda-cccl-11-8 (--purge):
 dependency problems - not removing
Errors were encountered while processing:
 cuda-cccl-11-8
(Reading database ... 120880 files and directories currently installed.)
Removing cuda-command-line-tools-11-8 (11.8.0-1) ...
(Reading database ... 120878 files and directories currently installed.)
Removing cuda-compat-11-8 (520.61.05-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/loc

In [12]:
!apt-get -y install cuda-toolkit-10-2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cuda-toolkit-10-2


In [29]:
%%cuda --name pcap.cu

#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION

#include <stdio.h>
#include <stdlib.h>
#include "stb_image.h"
#include "stb_image_write.h"

#define HISTOGRAM_SIZE 256  // Define the histogram size

// CUDA kernel for calculating histogram
__global__ void calculateHistogram(const unsigned char* input, int width, int height, int* histogram) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        atomicAdd(&histogram[input[y * width + x]], 1);
    }
}

// CUDA kernel for histogram equalization
__global__ void equalizeHistogram(const unsigned char* input, int width, int height, unsigned char* output, float* cdf, int totalPixels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int pixelValue = input[y * width + x];
        output[y * width + x] = static_cast<unsigned char>(255.0f * cdf[pixelValue]);
    }
}

// CUDA kernel for RGB to grayscale conversion
__global__ void rgbToGrayscale(const unsigned char* rgbImage, int width, int height, unsigned char* grayscaleImage) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int offset = (y * width + x) * 3;
        unsigned char r = rgbImage[offset];
        unsigned char g = rgbImage[offset + 1];
        unsigned char b = rgbImage[offset + 2];

        grayscaleImage[y * width + x] = static_cast<unsigned char>(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

int main() {
    // Load the input image (RGB)
    int width, height, channels;
    unsigned char* input_image = stbi_load("Lena.jpeg", &width, &height, &channels, 0);

    if (input_image == NULL) {
        printf("Failed to load the input image.\n");
        return 1;
    }

    // CUDA variables
    unsigned char* d_inputImage;
    cudaMalloc((void**)&d_inputImage, width * height * channels * sizeof(unsigned char));
    cudaMemcpy(d_inputImage, input_image, width * height * channels * sizeof(unsigned char), cudaMemcpyHostToDevice);

    // Convert RGB to grayscale
    unsigned char* d_grayscaleImage;
    cudaMalloc((void**)&d_grayscaleImage, width * height * sizeof(unsigned char));

    dim3 rgbToGrayscaleBlockSize(16, 16);
    dim3 rgbToGrayscaleGridSize((width + rgbToGrayscaleBlockSize.x - 1) / rgbToGrayscaleBlockSize.x, (height + rgbToGrayscaleBlockSize.y - 1) / rgbToGrayscaleBlockSize.y);

    rgbToGrayscale<<<rgbToGrayscaleGridSize, rgbToGrayscaleBlockSize>>>(d_inputImage, width, height, d_grayscaleImage);
    cudaDeviceSynchronize();

    // Calculate histogram for grayscale image
    int histogram[HISTOGRAM_SIZE] = {0};
    int* d_histogram;
    cudaMalloc((void**)&d_histogram, HISTOGRAM_SIZE * sizeof(int));

    dim3 histogramBlockSize(16, 16);
    dim3 histogramGridSize((width + histogramBlockSize.x - 1) / histogramBlockSize.x, (height + histogramBlockSize.y - 1) / histogramBlockSize.y);

    calculateHistogram<<<histogramGridSize, histogramBlockSize>>>(d_grayscaleImage, width, height, d_histogram);
    cudaDeviceSynchronize();

    // Copy histogram data back to host if needed
    cudaMemcpy(histogram, d_histogram, HISTOGRAM_SIZE * sizeof(int), cudaMemcpyDeviceToHost);

    // Calculate cumulative distribution function (CDF)
    float cdf[HISTOGRAM_SIZE];
    cdf[0] = static_cast<float>(histogram[0]) / (width * height);
    for (int i = 1; i < HISTOGRAM_SIZE; ++i) {
        cdf[i] = cdf[i - 1] + static_cast<float>(histogram[i]) / (width * height);
    }

    // Copy CDF to device
    float* d_cdf;
    cudaMalloc((void**)&d_cdf, HISTOGRAM_SIZE * sizeof(float));
    cudaMemcpy(d_cdf, cdf, HISTOGRAM_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    // Equalize histogram for grayscale image
    unsigned char* d_equalizedImage;
    cudaMalloc((void**)&d_equalizedImage, width * height * sizeof(unsigned char));

    dim3 equalizeBlockSize(16, 16);
    dim3 equalizeGridSize((width + equalizeBlockSize.x - 1) / equalizeBlockSize.x, (height + equalizeBlockSize.y - 1) / equalizeBlockSize.y);

    equalizeHistogram<<<equalizeGridSize, equalizeBlockSize>>>(d_grayscaleImage, width, height, d_equalizedImage, d_cdf, width * height);
    cudaDeviceSynchronize();

    // Copy the result back to the host if needed
    unsigned char* h_equalizedImage = new unsigned char[width * height];
    cudaMemcpy(h_equalizedImage, d_equalizedImage, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost);

    // Save the equalized image (replace with actual saving code)
    stbi_write_jpg("output_equalized.jpg", width, height, 1, h_equalizedImage, 100);

    // Clean up resources
    stbi_image_free(input_image);
    delete[] h_equalizedImage;
    cudaFree(d_inputImage);
    cudaFree(d_grayscaleImage);
    cudaFree(d_histogram);
    cudaFree(d_cdf);
    cudaFree(d_equalizedImage);

    return 0;
}


'File written in /content/src/pcap.cu'

In [30]:
!nvcc -o /content/src/pcap.out /content/src/pcap.cu -I.

./stb_image.h(4262): warning #550-D: variable "old_limit" was set but never used

./stb_image.h(5184): warning #550-D: variable "idata_limit_old" was set but never used

./stb_image.h(6971): warning #550-D: variable "out_size" was set but never used

./stb_image.h(6972): warning #550-D: variable "delays_size" was set but never used



In [31]:
!/content/src/pcap.out